In [ ]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

In [ ]:
#help(tf.losses.Reduction)
#help(tf.reduce_mean)

In [ ]:
OUTPUT_DIR = './tmp/rnn'
OUTPUT_DIR_TB = './tmp'
datadir = './data'
is_tensorboard = True

In [ ]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

In [ ]:
import p8_util
filename_dataset=None
x_train, x_valid, x_test, y_train, y_valid, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset, dataset_type='MNIST')

In [ ]:
x_train.shape

In [ ]:
import p8_util_config
import NNAdaNetBuilder

#-----------------------------------------------------------------
# Get the number of convolutional layers for CNN network
# This will fixe the equivalent parameter of AdaNet num layers.
#-----------------------------------------------------------------
layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['rnn_layer_num']
print("\n Number of layers= {}".format(layer_num))
oNNAdaNetBuilder = NNAdaNetBuilder.NNAdaNetBuilder(p8_util_config.dict_adanet_config, num_layers=layer_num)

In [ ]:
oNNAdaNetBuilder.show()

In [ ]:
# Parameters that will be provided to model_fn
import p8_util_config    X_seq = tf.unstack(X, axis=1) # shape: [batch_size, i, n_inputs], total num of i = n_steps

nn_type = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']


params = {'net_builder':oNNAdaNetBuilder, 'nn_type':nn_type}
print("\n NN Type= {}".format(nn_type))
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)  

classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)

In [ ]:
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train, p8_util_config.NUM_EPOCHS\
                          , tuple_dimension=tuple_dimension\
                          , batch_size=p8_util_config.BATCH_SIZE),
        max_steps=p8_util_config.MAX_STEPS),
    
    eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test, y_test, p8_util_config.NUM_EPOCHS\
                           , tuple_dimension=tuple_dimension\
                           , batch_size=p8_util_config.BATCH_SIZE),
        steps=None,
        throttle_secs=1))
end_time = time.time()

In [8]:
import tensorflow as tf
import numpy as np

X_data = np.array([
# steps   1st     2nd       3rd
        [[1, 2], [7, 8], [13, 14]],   # first batch
        #  <-- n_input -->
        [      [3, 4],       [9, 10], [15, 16]],  # second batch
        [      [5, 6],       [11, 12], [17, 18]], # third batch
        [      [1, 0],       [1, 12], [7, 8]]     # fourth batch
])
#help(tf.unstack)
X_data.shape
n_inputs = X_data.shape[2]
n_steps = X_data.shape[1]
n_inputs, n_steps

(2, 3)

In [6]:
# hyperparameters
n_neurons = 8

# parameters
n_inputs = X_data.shape[2]
n_steps = X_data.shape[1]

# rnn model
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seq = tf.unstack(X, axis=1) # shape: [batch_size, i, n_inputs], total num of i = n_steps



#XT_data = tf.transpose(X_data, [1, 0, 2]) 
#X_seq = tf.unstack(XT_data, axis=1) # shape: [batch_size, i, n_inputs], total num of i = n_steps


cell = tf.keras.layers.SimpleRNNCell(units=n_neurons)
# Here under, a static graph is created with n_neurons cells.
output, state = tf.nn.static_rnn(cell, X_seq, dtype=tf.float32)
X_seq = tf.unstack(X, axis=1) # shape: [batch_size, i, n_inputs], total num of i = n_steps

output_st = tf.stack(output, axis=1)


# initialize the variables
init = tf.global_variables_initializer()

# train
with tf.Session() as sess:
    sess.run(init)
    # X, input shape: (batch_size, time_step_size, input_vec_size)
    #X_data = tf.transpose(X_data, [1, 0, 2])  # permute time_step_size and batch_size
    feed_dict = {X: X_data}
    
    # print the shape
    if True :
        
        X_seq_shape = sess.run(tf.shape(X_seq), feed_dict=feed_dict)
        output_shape = sess.run(tf.shape(output), feed_dict=feed_dict)
        state_shape = sess.run(tf.shape(state), feed_dict=feed_dict)
        output_st_shape = sess.run(tf.shape(output_st), feed_dict=feed_dict)

        print('\nX_seq shape [batch_size, n_steps, n_inputs]: ', X_seq_shape)
        print('\nOutput shape [batch_size, n_neurons]: ', output_shape)
        print('\nState shape [batch_size, n_neurons]: ', state_shape)
        print('\nOutput_st shape [batch_size, n_steps, n_neurons]: ', output_st_shape)
    
    output_eval, state_eval = sess.run([output, state], feed_dict=feed_dict)
    
    print('\nIs the output of X2 equals to the state? ', np.array_equal(output_eval[2], state_eval))


X_seq shape [batch_size, n_steps, n_inputs]:  [3 4 2]

Output shape [batch_size, n_neurons]:  [3 4 8]

State shape [batch_size, n_neurons]:  [4 8]

Output_st shape [batch_size, n_steps, n_neurons]:  [4 3 8]

Is the output of X2 equals to the state?  True


In [ ]:
print(state_eval)

In [ ]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR_TB)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')